In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import ssl
import pandas as pd
import urllib.request
from PIL import Image
import time
import shutil
import requests



In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [15]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT DISTINCT ?country  ?capital ?country_name ?capital_name ?country_flag ?country_abstract
    WHERE {
      ?country a yago:WikicatCountries , dbo:Country.
      ?country dbo:capital ?capital.
      ?country rdfs:label ?country_name.
      ?capital rdfs:label ?capital_name.
      ?country dbo:thumbnail ?country_flag.
      ?country dbo:abstract ? ?country_abstract.

      FILTER(lang(?country_name) = 'en')
      FILTER(lang(?capital_name) = 'en')
      FILTER(lang(?country_abstract) = 'en')

    } 
    
    ORDER BY ?country
        
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results["results"].keys())

d = {
     "Country_uri" : [],
     "Country_name" : [],
     "Country_flag" : [], 
     "Capital_uri" : [], 
     "Capital_name" : [], 
     "Country_abstract" : []
    }

for result in results["results"]["bindings"]:
  d["Country_uri"].append(result["country"]["value"])
  d["Country_name"].append(result["country_name"]["value"])
  d["Country_flag"].append(result["country_flag"]["value"])
  d["Capital_uri"].append(result["capital"]["value"])
  d["Capital_name"].append(result["capital_name"]["value"])
  d["Country_abstract"].append(result["country_abstract"]["value"])

df_cities = pd.DataFrame(d)

df_cities["Country_flag"][74]



dict_keys(['distinct', 'ordered', 'bindings'])


"http://commons.wikimedia.org/wiki/Special:FilePath/Flag_of_Côte_d'Ivoire.svg?width=300"

In [4]:
#Printing random country with capital and flag

row = df_cities.sample(ignore_index=True).iloc[0]
country_name = row.loc["Country_name"]
country_capital = row["Capital_name"]
flag_link = row["Country_flag"]

print(f"Pays : {country_name}")
print(f"Capitale : {country_capital}")
  
urllib.request.urlretrieve(flag_link, "temp_flag.png")
  
img = Image.open("temp_flag.png")
img.show()


Pays : Albania
Capitale : Tirana


In [5]:
from urllib.parse import urlsplit, urlunsplit, quote

def iri2uri(iri):
    """
    Convert an IRI to a URI
    """
    uri = ''
    if isinstance(iri, str):
        (scheme, netloc, path, query, fragment) = urlsplit(iri)
        scheme = quote(scheme)
        netloc = netloc.encode('idna').decode('utf-8')
        path = quote(path)
        query = quote(query)
        fragment = quote(fragment)
        uri = urlunsplit((scheme, netloc, path, query, fragment))

    return uri

In [6]:
# Downloading all flags for all countries

done = [] # for coutries that appear twice or more in the dataset (multiple capitals)

for index, row in df_cities.iterrows():
    time.sleep(0.5)
    country = row["Country_name"]
    if country not in done:
        try :
            url = row["Country_flag"]
            urllib.request.urlretrieve(url, "./country_flags/" + country + "_flag.png")
            done.append(country)
        except:
            print(country)

Ivory Coast
Ivory Coast
Spain


We can see that we have some problems dealing with Ivory Coast and Spain, until now I have not find a solution so I'll just hardcode those two by downloading manually the flags...